In [2]:
import sounddevice as sd
import numpy as np
import PySimpleGUI as sg
import threading
from queue import Queue

# Get a list of input devices
devices = sd.query_devices()
input_devices = [device for device in devices if device['max_input_channels'] > 0]

# Function to convert the device info into a string for display
def device_info_to_string(device):
    return f"{device['name']} (Index: {device['index']})"

# Define the GUI layout
layout = [
    [sg.Text("Select a Microphone:")],
    [sg.Listbox([device_info_to_string(device) for device in input_devices], size=(60, 6), key='device_list')],
    [sg.Button('Start Preview'), sg.Button('Stop Preview')],
    [sg.Text("Microphone Sound Level:")],
    [sg.ProgressBar(100, orientation='h', size=(20, 20), key='sound_level')],
]

# Create GUI window
window = sg.Window('Microphone Sound Level Preview', layout)

# Callback function for audio recording
def audio_callback(indata, frames, time, status, q):
    q.put(indata.copy())

# Function for recording audio on a separate thread
def record_audio(q, stop_event, device_index):
    with sd.InputStream(callback=lambda *args: audio_callback(*args, q), samplerate=44100, device=device_index):
        while not stop_event.is_set():
            sd.sleep(100)

stop_event = threading.Event()
recording_thread = None
audio_queue = Queue()

# Event loop
while True:
    event, values = window.read(timeout=10)
    if event == sg.WIN_CLOSED:
        break
    elif event == 'Start Preview':
        selected_device_str = values['device_list'][0]
        selected_device_index = int(selected_device_str.split("Index: ")[1].split(")")[0])
        stop_event.clear()
        recording_thread = threading.Thread(target=record_audio, args=(audio_queue, stop_event, selected_device_index))
        recording_thread.start()
    elif event == 'Stop Preview':
        stop_event.set()
        if recording_thread is not None:
            recording_thread.join()

    # Update sound level progress bar
    if not audio_queue.empty():
        audio_data = audio_queue.get()
        sound_level = int(np.mean(np.abs(audio_data)) * 100)
        window['sound_level'].update(sound_level)

# Close GUI window
window.close()

In [1]:
import sounddevice as sd
print(sd.query_devices())


< 0 HD-Audio Generic: DELL U2417H (hw:0,3), ALSA (0 in, 2 out)
  1 HD-Audio Generic: DELL P2418HZm (hw:0,7), ALSA (0 in, 2 out)
  2 HD-Audio Generic: DELL U2417H (hw:0,8), ALSA (0 in, 2 out)
> 3 HD-Audio Generic: ALC257 Analog (hw:1,0), ALSA (2 in, 2 out)
  4 acp: - (hw:2,0), ALSA (2 in, 0 out)
  5 ThinkPad USB-C Dock Gen2 USB Au: Audio (hw:3,0), ALSA (1 in, 2 out)
  6 JOUNIVO JV601: USB Audio (hw:4,0), ALSA (2 in, 0 out)
  7 P2418HZm: USB Audio (hw:5,0), ALSA (2 in, 2 out)
  8 hdmi, ALSA (0 in, 2 out)
